In [ ]:
!pip install transformers

In [ ]:
!pip install torchreid

In [ ]:
!pip install pytorch_lightning

##Библиотеки и конфиг


In [220]:
import json
from google.colab import files
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
import pandas as pd
import os
import tokenizers
import string
import transformers
import torch.nn as nn
from torch.nn import functional as F
import re
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSequenceClassification
from transformers import AutoConfig, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import numpy as np
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.optim import lr_scheduler
from sklearn import model_selection
from sklearn import metrics
from torchreid import utils
from tqdm.autonotebook import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

<ipython-input-220-63b52b00f0ca>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

In [106]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 5
PATH = "DeepPavlov/rubert-base-cased"
MODEL = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")
TOKENIZER = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
device = torch.device("cuda") if torch.cuda.is_available else torch.device("cpu")

In [5]:
device

device(type='cuda')

##Датасет

train

In [6]:
df = pd.read_json('/content/train.json', orient='columns')

In [7]:
df['ans'] = df['extracted_part'].apply(lambda x : x['text'][0])
df['start'] = df['extracted_part'].apply(lambda x : x['answer_start'][0])
df['end'] = df['extracted_part'].apply(lambda x : x['answer_end'][0])
df['target'] = df['ans'].apply(lambda x : 0 if x == '' else 1)
del df['extracted_part']

In [8]:
df.head()

,id,text,label,ans,start,end,target
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...,1279,1343,1
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...,1222,1318,1
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1297,1343,1
3,184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1304,1350,1
4,211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1302,1348,1


train question-answering

In [9]:
df_qa = df[df.ans != '']

test

In [10]:
df_test = pd.read_json('/content/test.json', orient='columns')

In [11]:
df_test.head()

,id,text,label
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта


##Классификация

обучение

In [12]:
def process_data_classification(text, sentiment, tokenizer, max_len):
    tok_text = tokenizer.encode_plus(text, return_offsets_mapping=True)

    input_ids_orig = tok_text.input_ids[1:-1]

    tok_sentiment = tokenizer.encode_plus(sentiment, return_offsets_mapping=True)['input_ids'][1:-1]

    input_ids = [101] + tok_sentiment + [102] + input_ids_orig + [102]

    input_ids=input_ids[:512] #ограничеваем тонены

    token_type_ids = [0] + [1]*len(tok_sentiment) + [0] + [1] * (len(input_ids_orig) + 1)

    token_type_ids = token_type_ids[:512] #ограничеваем тонены

    mask = [1] * len(token_type_ids)


    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
    }

In [13]:
class TweetDatasetClass:
    def __init__(self, text, sentiment, ans):
        self.text = text
        self.sentiment = sentiment
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
        self.ans = ans
    
    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        data = process_data_classification(
            self.text[item],
            self.sentiment[item],
            self.tokenizer,
            self.max_len,
        )

        return {
            'input_ids': torch.tensor(data["ids"], dtype=torch.long),
            'attention_mask': torch.tensor(data["mask"], dtype=torch.long),
            'targets': torch.tensor(self.ans[item], dtype=torch.long)
        }

In [16]:
class BertClassifier:

    def __init__(self, n_classes=2, epochs=1, model_save_path='/content/bert.pt'):
        self.model = AutoModelForSequenceClassification.from_pretrained(PATH)
        self.device = device
        self.model_save_path=model_save_path
        self.max_len = 512
        self.epochs = epochs
        self.model.classifier = torch.nn.Linear(768, n_classes)
        self.model.to(device)
    
    def preparation(self, train_set, valid_set):
        self.train_set = train_set
        self.valid_set = valid_set

        self.train_loader = DataLoader(self.train_set, batch_size=2, shuffle=True)
        self.valid_loader = DataLoader(self.valid_set, batch_size=2, shuffle=True)

        self.optimizer = AdamW(self.model.parameters(), lr=2e-5, correct_bias=False)
        self.scheduler = get_linear_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=0,
                num_training_steps=len(self.train_loader) * self.epochs
            )
        self.loss_fn = torch.nn.CrossEntropyLoss().to(self.device)
            
    def fit(self):
        self.model = self.model.train()
        losses = []
        correct_predictions = 0

        for data in self.train_loader:
            input_ids = data["input_ids"].to(self.device)
            attention_mask = data["attention_mask"].to(self.device)
            targets = data["targets"].to(self.device)

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
                )

            preds = torch.argmax(outputs.logits, dim=1)
            loss = self.loss_fn(outputs.logits, targets)

            correct_predictions += torch.sum(preds == targets)

            losses.append(loss.item())
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.scheduler.step()
            self.optimizer.zero_grad()

        train_acc = correct_predictions.double() / len(self.train_set)
        train_loss = np.mean(losses)
        return train_acc, train_loss
    
    def eval(self):
        self.model = self.model.eval()
        losses = []
        correct_predictions = 0

        with torch.no_grad():
            for data in self.valid_loader:
                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                targets = data["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                    )

                preds = torch.argmax(outputs.logits, dim=1)
                loss = self.loss_fn(outputs.logits, targets)
                correct_predictions += torch.sum(preds == targets)
                losses.append(loss.item())
        
        val_acc = correct_predictions.double() / len(self.valid_set)
        val_loss = np.mean(losses)
        return val_acc, val_loss
    
    def train(self):
        best_accuracy = 0
        for epoch in range(self.epochs):
            print(f'Epoch {epoch + 1}/{self.epochs}')
            train_acc, train_loss = self.fit()
            print(f'Train loss {train_loss} accuracy {train_acc}')

            val_acc, val_loss = self.eval()
            print(f'Val loss {val_loss} accuracy {val_acc}')
            print('-' * 10)

            if val_acc > best_accuracy:
                torch.save(self.model, self.model_save_path)
                best_accuracy = val_acc

        self.model = torch.load(self.model_save_path)
    

In [17]:
classifier = BertClassifier(
        n_classes=2,
        epochs=3,
        model_save_path='/content/bert.pt'
)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [18]:
tmp = TweetDatasetClass(
        df.text.values,
        sentiment=df.label.values,
        ans=df.target.values,
    )

train_idx, val_idx = train_test_split(range(len(tmp)), test_size=0.1, random_state=42)
train_set = Subset(tmp, train_idx)
valid_set = Subset(tmp, val_idx)

In [19]:
classifier.preparation(
        train_set,
        valid_set
    )

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
classifier.train()

Epoch 1/3
Train loss 0.2200180208248325 accuracy 0.9524397776405189
Val loss 0.0005827262628978739 accuracy 1.0
----------
Epoch 2/3
Train loss 0.015724677211255766 accuracy 0.9975293390982088
Val loss 8.012930742956491e-05 accuracy 1.0
----------
Epoch 3/3
Train loss 0.00434446001861404 accuracy 0.9993823347745523
Val loss 0.034143004419941766 accuracy 0.9944444444444445
----------


классификация

In [24]:
class TweetDatasetTest:
    def __init__(self, text, sentiment):
        self.text = text
        self.sentiment = sentiment
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
    
    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        data = process_data_classification(
            self.text[item],
            self.sentiment[item],
            self.tokenizer,
            self.max_len,
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
        }

In [25]:
test = TweetDatasetTest(
        text=df_test.text.values,
        sentiment=df_test.label.values
    )

In [37]:
ans = []

for i in tqdm(range(test.__len__())):
  d = test.__getitem__(i)
  ids = d["ids"].to(device, dtype=torch.long).view(1, -1)
  mask = d["mask"].to(device, dtype=torch.long).view(1, -1)


  classifier.model.eval()

  outputs = classifier.model(
                input_ids=ids,
                attention_mask=mask
                )

  preds = torch.argmax(outputs.logits, dim=1)
  ans.append(preds.item())

100%|██████████| 318/318 [00:12<00:00, 25.41it/s]


In [38]:
df_test['ans'] = ans


In [39]:
df_test.head()

,id,text,label,ans
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта,1
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,1
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта,1
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта,1
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта,1


In [40]:
df_test.ans.value_counts()

1    264
0     54
Name: ans, dtype: int64

In [105]:
df_test.to_csv('/content/df_test.csv', index= False) 

##Question-answering

In [78]:
df = df[df.ans != '']

In [79]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len, start, end):
    idx0 = start-1
    idx1 = end-1

    char_targets = [0] * len(tweet)
    if idx0 != -1 and idx1 != -1:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode_plus(tweet, return_offsets_mapping=True)

    input_ids_orig = tok_tweet.input_ids[1:-1]
    tweet_offsets = tok_tweet.offset_mapping[1:-1]

    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)

    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]


    tok_sentiment = tokenizer.encode_plus(sentiment, return_offsets_mapping=True)['input_ids'][1:-1]

    input_ids = [101] + tok_sentiment + [102] + input_ids_orig + [102]

    input_ids=input_ids[:512]

    token_type_ids = [0] + [0]*len(tok_sentiment) + [0] + [1] * (len(input_ids_orig) + 1)

    token_type_ids = token_type_ids[:512]

    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] + [(0, 0)] * len(tok_sentiment) + [(0, 0)] + tweet_offsets + [(0, 0)]

    tweet_offsets = tweet_offsets[:512]

    targets_start += 2 + len(tok_sentiment)
    targets_end += 2 + len(tok_sentiment)


    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }

In [80]:
class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text, start, end):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
        self.start = start 
        self.end = end
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len,
            self.start[item],
            self.end[item]
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

In [81]:
tmp = TweetDataset(
        tweet=df.text.values,
        sentiment=df.label.values,
        selected_text=df.ans.values,
        start = df.start.values,
        end = df.end.values,
    )

In [82]:
train_idx, val_idx = train_test_split(range(len(tmp)), test_size=0.1, random_state=42)

train_dataset = Subset(tmp, train_idx)
val_dataset = Subset(tmp, val_idx)

In [83]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    num_workers=1
)

valid_data_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=VALID_BATCH_SIZE,
    num_workers=1
)

In [84]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()

    n = start_logits.shape[0]

    start = torch.zeros(n, 512).to(device)
    for i in range(n):
      start[i][start_positions[i]] = 1

    end = torch.zeros(n, 512).to(device)
    for i in range(n):
      end[i][end_positions[i]] = 1

    start_loss = loss_fct(start_logits, start)
    end_loss = loss_fct(end_logits, end)
    total_loss = (start_loss + end_loss)
    return total_loss

In [162]:
class TweetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = MODEL
        # self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        out = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        ).last_hidden_state


        # out = self.drop_out(out)

        logits = self.l0(out)
        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [163]:
model = TweetModel()
model.to(device)

TweetModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [164]:
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    losses = utils.AverageMeter()

    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for bi, d in enumerate(tk0):

        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        offsets = d["offsets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)

        model.zero_grad()
        outputs_start, outputs_end = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids,
        )

        loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
        loss.backward()
        optimizer.step()
        scheduler.step()

        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()

        losses.update(loss.item(), ids.size(0))

In [165]:
def eval_fn(data_loader, model, device):
    model.eval()
    losses = utils.AverageMeter()
    
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader))
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].numpy()

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)

            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()

            losses.update(loss.item(), ids.size(0))
            tk0.set_postfix(loss=losses.avg)
    
    print(f"Loss = {losses.avg}")
    return losses.avg

In [166]:
def run():
    num_train_steps = int(len(tmp) / TRAIN_BATCH_SIZE * EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

    es = EarlyStopping(monitor="train_loss", patience=2, mode="max")
    
    for epoch in range(3):
        train_fn(train_data_loader, model, optimizer, device, scheduler=scheduler)
        loss = eval_fn(valid_data_loader, model, device)
        print(f"Loss = {loss}")


In [168]:
run()

  0%|          | 0/168 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Loss = 1.028564218133688
Loss = 1.028564218133688


  0%|          | 0/168 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Loss = 1.0218101909508308
Loss = 1.0218101909508308


  0%|          | 0/168 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Loss = 1.0218101909508308
Loss = 1.0218101909508308


In [91]:
torch.save(model.state_dict(), '/content/qa.pt')

In [188]:
def search(s, t):
    s1 = s
    t1 = t
    s = s.split()
    t = t.split()
    check = ''.join(i for i in t)
    try:
        for i in range(len(s)):
            tmp = []
            j = 0
            k = 0
            while j < len(t):
                if s[i + k] == t[j]:
                    tmp.append(t[j])
                elif t[j] in s[i + k]:
                    p = t[j]
                    cnt1 = 0
                    while p in s[i + k]:
                        if p == s[i + k]:
                            tmp.append(s[i + k])
                            break
                        cnt1 += 1
                        p += t[j + cnt1]
                    j += cnt1
                else:
                    break
                k += 1
                j += 1
            if check == ''.join(i for i in tmp):
                return ' '.join(i for i in tmp)
    except IndexError:
        s = s1.split('.')
        for i in s:
            if ''.join(x for x in t1.split()) in ''.join(x for x in i.split()):
                return i.lstrip() + '.'
        return ''
    s = s1.split('.')
    for i in s:
      if ''.join(x for x in t1.split()) in ''.join(x for x in i.split()):
        return i.lstrip() + '.'
    return ''

In [131]:
def process_data_test(tweet, sentiment, tokenizer, max_len):    
    tok_tweet = tokenizer.encode_plus(tweet, return_offsets_mapping=True)

    input_ids_orig = tok_tweet.input_ids[1:-1]
    tweet_offsets = tok_tweet.offset_mapping[1:-1]


    tok_sentiment = tokenizer.encode_plus(sentiment, return_offsets_mapping=True)['input_ids'][1:-1]

    input_ids = [101] + tok_sentiment + [102] + input_ids_orig + [102]

    input_ids=input_ids[:512] 

    token_type_ids = [0] + [0]*len(tok_sentiment) + [0] + [1] * (len(input_ids_orig) + 1)

    token_type_ids = token_type_ids[:512]

    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] + [(0, 0)] * len(tok_sentiment) + [(0, 0)] + tweet_offsets + [(0, 0)]

    tweet_offsets = tweet_offsets[:512]


    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'orig_tweet': tweet,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }

In [132]:
class TweetDatasetTest:
    def __init__(self, tweet, sentiment):
        self.tweet = tweet
        self.sentiment = sentiment
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data_test(
            self.tweet[item],
            self.sentiment[item],
            self.tokenizer,
            self.max_len,
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

In [133]:
tmp = TweetDatasetTest(
        tweet=df_test.text.values,
        sentiment=df_test.label.values
    )

In [189]:
sentiment_ans = []
for i in tqdm(range(tmp.__len__())):

  d = tmp.__getitem__(i)

  ids = d["ids"]
  token_type_ids = d["token_type_ids"]
  mask = d["mask"]
  sentiment = d["sentiment"]
  orig_tweet = d["orig_tweet"]
  offsets = d["offsets"]

  ids = ids.to(device, dtype=torch.long)
  token_type_ids = token_type_ids.to(device, dtype=torch.long)
  mask = mask.to(device, dtype=torch.long)

  model.eval()

  outputs_start, outputs_end = model(
      ids=ids.view(1, -1),
      mask=mask.view(1, -1),
      token_type_ids=token_type_ids.view(1, -1),
  )
  t = TOKENIZER.decode(ids[outputs_start.argmax():outputs_end.argmax()+1])
  s = orig_tweet
  sentiment_ans.append(search(s, t))

  0%|          | 0/318 [00:00<?, ?it/s]

In [190]:
df_test['sentiment_ans'] = sentiment_ans

In [191]:
df_test.to_csv('/content/df_test4.csv', index= False) 

In [198]:
df_test.loc[df_test['ans'] == 0, 'sentiment_ans'] = ''

In [201]:
start = []
end = []

for i in tqdm(range(318)):
  start.append(df_test['text'].loc[i].find(df_test['sentiment_ans'].loc[i]))
  end.append(start[i]+len(df_test['sentiment_ans'].loc[i]))

  0%|          | 0/318 [00:00<?, ?it/s]

In [206]:
df_test['start'] = start
df_test['end'] = end

In [212]:
df_test.columns

Index(['id', 'text', 'label', 'ans', 'sentiment_ans', 'answer_start',
       'answer_end'],
      dtype='object')

In [210]:
df_test = df_test.rename(columns={'start': 'answer_start', 'end': 'answer_end'})

In [229]:
data = df_test.to_dict(orient='records')
result = []

for row in data:
    extracted_part = {
        "text": [row["sentiment_ans"]],
        "answer_start": [row["answer_start"]],
        "answer_end": [row["answer_end"]]
    }
    json_row = {
        "id": row["id"],
        "text": row["text"],
        "label": row["label"],
        "extracted_part": extracted_part
    }
    result.append(json_row)

with open('predictions.json', 'w') as f:
    json.dump(result, f, ensure_ascii=False)

files.download('predictions.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>